In [20]:
using VQE, LinearAlgebra
using Printf
using HDF5 

import Random, Distributions

using PyPlot

In [21]:
# load the SK hard instance data

# number of spins
N = 12

PATH = raw"/home/ubuntu/aqc_QAOA/SpinFluctuations.jl"
PATH_t = raw"/home/ubuntu/aqc_QAOA/transformation_hard_SK_instances"

#subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("/data/N_%i/", N);
folder_name_t = PATH_t * @sprintf("/data/N_%i/", N);

In [22]:
seed = 10103
#100774
#10103
#124718
#101738
#100052

#spin_idx = 2
file_name = folder_name * @sprintf("hard_random_SK_instance_N_%i_seed_%i.h5", N , seed)
file_name_t = folder_name_t * @sprintf("transformed_hard_SK_instance_N_%i_seed_%i.h5", N , seed)

gs_energy = h5read(file_name, "ground_state_energy") 
J_mat = h5read(file_name, "couplings"); 
local_fields = h5read(file_name, "local_fields")
mf_problem = Problem(0, local_fields, J_mat);
println(gs_energy)

gs_energy_t = h5read(file_name_t, "ground_state_energy")
J_mat_t = h5read(file_name_t, "couplings"); 
local_fields_t = h5read(file_name_t, "local_fields")
mf_problem_t = Problem(0, local_fields_t, J_mat_t);
mf_problem_t
#println(gs_energy_t)
#println(J_mat_t)
#println(local_fields_t)
println(mf_problem_t)

-6.621548884190914
Problem
  num_qubits: Int64 12
  num_layers: Int64 0
  local_fields: Array{Real}((12,)) Real[-3.487046747337776, -0.8806716417717356, -3.8960556820418963, 3.8168877439625866, -5.0438929786846805, -1.9671340854822044, -0.1152917411734118, -2.2284985262254167, -1.4622957743414886, -1.393290878798267, -0.6244913490750017, 1.741156139263947]
  couplings: Array{Real}((12, 12)) Real[0.0 0.15332470251411254 … 5.476511645685741 2.23125092381722; 0.15332470251411254 0.0 … -1.56460601704686 0.49990345968646316; … ; 5.476511645685741 -1.56460601704686 … 0.0 -6.482658138404204; 2.23125092381722 0.49990345968646316 … -6.482658138404204 0.0]
  edges: Array{CartesianIndex{2}}((132,))
  driver: YaoBlocks.ConstGate.XGate X



In [23]:
N = 4
σ2 = 1.0

Random.seed!(1)
J = rand(Distributions.Normal(0, σ2), N, N) ./ sqrt(N) 
J[diagind(J)] .= 0.0
J = UpperTriangular(J)
J = J + transpose(J)
     
N = N-1
p = 2
SK_problem = Problem(p, J)
mf_problem = Problem(0, J);

SK_problem

Problem
  num_qubits: Int64 3
  num_layers: Int64 2
  local_fields: Array{Real}((3,)) Real[0.13375702345533, 0.0036930338557504432, 0.5303622417454488]
  couplings: Array{Real}((3, 3)) Real[0.0 0.5428970107716381 -1.3951405027746535; 0.5428970107716381 0.0 -0.9460077791129564; -1.3951405027746535 -0.9460077791129564 0.0]
  edges: Array{CartesianIndex{2}}((6,))
  driver: YaoBlocks.ConstGate.XGate X


In [25]:
circ_c = VQE.circuit(SK_problem)
num_params = VQE.nparameters(circ_c)
initial_params = vcat([0.5 for _ in 1:num_params])
#println(initial_params)
#initial_params = vcat([0.5 for _ in 1:2p])


learning_rate = 0.03
niter = 200

cost, i_params, probs, cost_history = optimize_parameters(SK_problem, initial_params; niter, learning_rate=learning_rate)

(2.8821530113636245, [-0.03824886512235429, -0.009374035821602613, 1.694327737944832, 1.0642424456422, 0.3659031199150343, 0.18896504978768686, 0.21280560179444982, 0.37149756539629736, 0.024389651380446468, 0.04429712930978625, 0.0021794650910858557, 1.4451226648148163, 0.6129916589363935, 0.7277189580307749, 0.7390950324981234, 0.7454511745340318, 0.8746509168982032, 1.559757776940473], [0.0002903114154316422, 0.00020070487059923106, 0.00012506095610588722, 0.5009683805949354, 0.49818344342240534, 9.799751871922998e-5, 0.00013206218155846372, 2.0390402445920224e-6], Any[-0.9381588982079099, -0.8762259048431159, -0.8090105931818747, -0.7366234676570417, -0.6593615930289107, -0.5777154292463639, -0.4923565173789368, -0.4041037626135555, -0.31386960250874507, -0.22259136141875316  …  2.880581559162221, 2.880787066747171, 2.880984001993082, 2.8811727535815663, 2.8813536922138745, 2.881527171466949, 2.8816935286075296, 2.881853085366366, 2.8820061486745203, 2.8821530113636245])